In [2]:
from __future__ import division 
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable

In [3]:
def parse_cfg(cfgfile):
  """
  - Take a config files adn returns a list of block. 
  - Each block describes a block in nn that need to be built
  - Block is in form of a dictionary 
  
  The attribute of blocks and their values are stores as key-val pairs in dict
  
  """

  # Saving the content of the cfg file in string format
  file = open(cfgfile, 'r')
  lines = file.read().split('\n')                   # store the lines in a list 
  lines = [x for x in lines if len(x) > 0]          # ignoreing empty lines
  lines = [x for x in lines if x[0] != '#']         # get rid of comments
  lines = [x.rstrip().lstrip() for x in lines]      # get rid of whitespce to the right and left of word

  #looping over the resultant lines to bet blocks
  block = {}
  blocks = []
  for line in lines:
    if line[0] == '[':                      # '[' this marks the start of a new block
      if len(block) != 0:                   # !block.empty  --> block is storing some values 
        blocks.append(block)                # add it to blocks list
        block = {}                          # re-init the block
      block['type'] = line[1:-1]
    else:
      key, value = line.split('=')
      block[key.rstrip()] = value.lstrip()
  blocks.append(block)

  return blocks




In [8]:
"""
What's an empty layer?
- 
"""
class EmptyLayer(nn.Module):
    def __init__(self):
        super(EmptyLayer, self).__init__()

def create_modules(blocks):
  net_info = blocks[0]                # capturing info about i/p & preprocessing
  module_list = nn.ModuleList()
  prev_filter = 3                   
  # used to track the no. of filter in the layer which the convolution layer is appled
  # has been initalized to 3 filters corresponding to RGB channels
  output_filters = []     # stores the number of o/p filters 

  # iterate over the list of blocks, and create a PyTorch module for each block
  for index, x in enumerate(blocks[1:]):
    module = nn.Sequential()
    
    """
    Convolution Layer
    - 
    """ 
    if (x["type"] == "convolutional"):
      #Get the info about the layer
      activation = x["activation"]
      try:
        batch_normalize = int(x["batch_normalize"])
        bias = False
      except:
        batch_normalize = 0
        bias = True
        
      filters= int(x["filters"])
      padding = int(x["pad"])
      kernel_size = int(x["size"])
      stride = int(x["stride"])
        
      if padding:
        pad = (kernel_size - 1) // 2
      else:
        pad = 0
        
      #Add the convolutional layer
      conv = nn.Conv2d(prev_filters, filters, kernel_size, stride, pad, bias = bias)
      module.add_module("conv_{0}".format(index), conv)
        
      #Add the Batch Norm Layer
      if batch_normalize:
        bn = nn.BatchNorm2d(filters)
        module.add_module("batch_norm_{0}".format(index), bn)
        
      #Check the activation. 
      #It is either Linear or a Leaky ReLU for YOLO
      if activation == "leaky":
        activn = nn.LeakyReLU(0.1, inplace = True)
        module.add_module("leaky_{0}".format(index), activn)
    

    """
    Route Layer / Shortcut Layer
    - Extract the values of the layer attribute 
    - Cat ti into an int and store in list
    """
    elif (x["type"] == "route"):
      x["layers"] = x["layers"].split(',')
      #Start  of a route
      start = int(x["layers"][0])
      #end, if there exists one.
      try:
        end = int(x["layers"][1])
      except:
        end = 0
      #Positive anotation
      if start > 0: 
        start = start - index
      if end > 0:
        end = end - index
      route = EmptyLayer()
      module.add_module("route_{0}".format(index), route)
      if end < 0:
        filters = output_filters[index + start] + output_filters[index + end]
      else:
        filters= output_filters[index + start]

      #shortcut corresponds to skip connection
      elif x["type"] == "shortcut":
        shortcut = EmptyLayer()
        module.add_module("shortcut_{}".format(index), shortcut)
  
